In [ ]:
from matplotlib.pyplot import figure, rc
from numpy import zeros, linspace
from numpy.typing import NDArray
from typing import Callable, Any
from itertools import pairwise
from numpy import floating

In [ ]:
NDArray = NDArray[floating[Any]]
Function = Callable[[NDArray], NDArray]

In [ ]:
def curve(D: NDArray, I: NDArray, n: int):
  G = D.reshape(-1, 1) ** range(n)
  A, B = (G.T @ G), (G.T @ I)

  L, U = zeros((n, n)), A
  L.flat[::(n + 1)] = 1

  for i, ii in pairwise(range(n)):
    L[ii:, i] = U[ii:, i] / U[i, i]
    U[ii:] -= L[ii:, i, None] * U[i]

  x, y = zeros(n), zeros(n)

  for i in range(n):
    y[i] = B[i] - (L[i] * y).sum()

  for i in reversed(range(n)):
    temp = y[i] - (U[i] * x).sum()
    x[i] = temp / U[i, i]

  return (G @ x)

In [ ]:
def integral(func: Function, a: int, b: int, n: int):
  def trapezio(D: NDArray, I: NDArray):
    diff = (D[1:] - D[:-1]).mean() / 2
    return diff * (2 * I.sum() - I[0] - I[-1])

  def simpson(D: NDArray, I: NDArray):
    diff = (D[1:] - D[:-1]).mean() / 3
    prt1, prt2 = I[1:-1:2].sum(), I[2:-2:2].sum()
    return diff * (I[0] + I[-1] + 4 * prt1 + 2 * prt2)

  I1 = func(D1 := linspace(a, b, 1000))
  I2 = func(D2 := linspace(a, b, n))
  I3 = curve(D2, func(D2), 10)
  F1 = (D2, I2, trapezio(D2, I2), simpson(D2, I2))
  F2 = (D2, I3, trapezio(D2, I3), simpson(D2, I3))

  return (D1, I1, F1, F2)

In [ ]:
def plotgfx(D1, I1, F1, F2):
  rc("font", family = "Arial", size = 10)
  rc("lines", aa = 1, lw = .5)

  fig = figure(figsize = (12, 3))
  axes = fig.subplots(1, 2); fig.dpi = 400

  for (axe, label, (D, I, R1, R2)) in zip(
      axes, ("Aproximado", "Ajustado"), (F1, F2)):
    axe.plot(D1, I1, label = "Original", c = "r")
    axe.plot(D, I, label = label, c = "k")
    axe.fill_between(D, I, color = "k", alpha = .3)
    axe.set_title(f"Trapézio: {R1}, Simpson: {R2}")
    axe.set_xlabel("Domínio da função")
    axe.set_ylabel("Imagem da função")
    axe.legend()

  fig.suptitle("Integralização da Função")
  fig.tight_layout(pad = 1)

In [ ]:
plotgfx(*integral(lambda x: x**(1/2), 1, 4, 5))
plotgfx(*integral(lambda x: x**(1/2), 1, 4, 7))
plotgfx(*integral(lambda x: 1/(x**(1/2)), 2, 14, 5))
plotgfx(*integral(lambda x: 1/(x**(1/2)), 2, 14, 7))